In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/ftei-dsw/projects/weather-gan

/home/ftei-dsw/projects/weather-gan


In [8]:
import torch
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from src.model import Generator, Discriminator
from src.trainer import WeatherGenerator
from src.data import DataModule
from src.utils import get_version
from src.configuration import Config
from src.trainer import training

# start training

In [9]:
Config()

Config(tensorboard_path='/home/ftei-dsw/data/weather-gan/tensorboard/', checkpoint_path='/home/ftei-dsw/data/weather-gan/checkpoints/', config_path='/home/ftei-dsw/data/weather-gan/config-files/', results_path='/home/ftei-dsw/data/weather-gan/results/', input_fname='/home/ftei-dsw/data/weather-gan/datasets/monthly_gfdl_historical.nc', target_fname='/home/ftei-dsw/data/weather-gan/datasets/daily_gfdl_historical.nc', train_start=1950, train_end=1986, valid_start=2000, valid_end=2014, test_start=2000, test_end=2014, model_name='weather-gan', num_variables=2, epochs=10, train_batch_size=32, test_batch_size=64, lr=0.0002, beta1=0.5, beta2=0.999, latent_dim=8, apply_dropout=False, generator_channels=64, generator_num_resblocks=5, generator_num_downsampling=2, discriminator_channels=32)

In [ ]:
training(Config(), enable_profiler=True)

# Dev